In [31]:
import tensorflow as tf
import os as os
import numpy as np
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten


In [32]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [33]:
image_exts = ['jpeg','jpg', 'bmp', 'png']
data_dir = "Rice_Image_Dataset\Train"

for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

In [34]:
data = tf.keras.utils.image_dataset_from_directory('Rice_Image_Dataset\Train',image_size = (64,64))
scaled_data = data.map(lambda x,y: (x/255, y))
batch = scaled_data.as_numpy_iterator().next()

Found 50000 files belonging to 5 classes.


In [35]:
train_size = int(len(scaled_data)*0.8)
val_size = int(len(scaled_data)*0.2)

train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)

In [36]:
train

<_TakeDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [40]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 31, 31, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 29, 29, 16)        2320      
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 14, 14, 16)        0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 3136)              0         
                                                                 
 dense_18 (Dense)            (None, 500)              

In [43]:
logdir='logs'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
1250/1250 [==============================] - 141s 113ms/step - loss: 0.0679 - accuracy: 0.9769 - val_loss: 0.0558 - val_accuracy: 0.9800
Epoch 2/20
1250/1250 [==============================] - 145s 116ms/step - loss: 0.0452 - accuracy: 0.9850 - val_loss: 0.0370 - val_accuracy: 0.9866
Epoch 3/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.0333 - accuracy: 0.9886 - val_loss: 0.0383 - val_accuracy: 0.9872
Epoch 4/20
1250/1250 [==============================] - 140s 112ms/step - loss: 0.0244 - accuracy: 0.9916 - val_loss: 0.0240 - val_accuracy: 0.9929
Epoch 5/20
1250/1250 [==============================] - 137s 110ms/step - loss: 0.0197 - accuracy: 0.9928 - val_loss: 0.0263 - val_accuracy: 0.9916
Epoch 6/20
1250/1250 [==============================] - 141s 112ms/step - loss: 0.0171 - accuracy: 0.9941 - val_loss: 0.0263 - val_accuracy: 0.9912
Epoch 7/20
1250/1250 [==============================] - 136s 109ms/step - loss: 0.0155 - accuracy: 0.9948 - val_

In [45]:
model.save('RiceClassifier.h5')
